In [ ]:
from pathlib import Path
import importlib
import open3d as o3d
import numpy as np
import cv2
import matplotlib.pyplot as plt
from src import object_render
importlib.reload(object_render)
from src import model_save_load
importlib.reload(model_save_load)
from src import object_detection
importlib.reload(object_detection)
from src import bbox_utils
importlib.reload(bbox_utils)


In [ ]:
data_dir = Path("./data/hammer")
mesh_file = data_dir / "object_convex_decomposition.obj"
mesh_o3d, mesh_trimesh, info = model_save_load.load_mesh(mesh_file, vis=False)

In [ ]:
object_render.get_render([1,1,0], 30, mesh_o3d, data_dir/"example")

In [ ]:
azimuth_samples = np.linspace(0,360,20,endpoint=False)
elevation_cos = np.linspace(-1,1,10,endpoint=True)
elevation_samples = (np.rad2deg(np.arccos(elevation_cos))).astype("int32")
rotation_samples = np.linspace(0, 360, 20,endpoint=False)


In [ ]:
elevation_samples

In [ ]:
for elevation in elevation_samples:
    if elevation==0 or elevation==180:
        for rot in rotation_samples:
            object_render.get_render([0,0,np.cos(np.radians(elevation))], rot, mesh_o3d,data_dir/"renders")
    else:
        for azimuth in azimuth_samples:
            for rot in rotation_samples:
                object_render.get_render([np.cos(np.radians(azimuth))*np.sin(np.radians(elevation)),np.sin(np.radians(azimuth))*np.sin(np.radians(elevation)),np.cos(np.radians(elevation))], rot, mesh_o3d,data_dir/"renders")


In [ ]:
render_dir = data_dir/"renders"
render_iter = render_dir.glob("render_*")
path_list= list(render_iter)
len(path_list)

In [ ]:
image = cv2.imread(next(render_iter))
bbox = object_detection.detect_object_with_opencv(image, vis =True)
croped_image = bbox_utils.crop_bbox_cv2(image,bbox)

# cv2.imshow('Matches', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
image = cv2.imread(data_dir/"mask.png")
image = bbox_utils.add_bbox_to_image(image, np.array([225, 158, 275, 222]))
bbox = bbox_utils.get_total_bounding_box(image,background_white=True)
croped_image_scene = bbox_utils.crop_bbox_cv2(image,bbox)

In [ ]:
plt.imshow(croped_image_scene)

In [ ]:

mask = np.all(croped_image_scene==np.array([200,200,200]), axis=-1)
croped_image_scene[mask] = [0, 0, 0]
plt.imshow(croped_image_scene) 

In [ ]:
image = cv2.imread(path_list[max_ind])
bbox = object_detection.detect_object_with_opencv(image, vis = False)
croped_image = bbox_utils.crop_bbox_cv2(image,bbox)
score, mask = comparison_score(croped_image, croped_image_scene, vis=True)
score



In [ ]:
comparison_score(croped_image, croped_image_scene)

In [ ]:
max_score = -np.inf
max_ind = -1
max_mask = None
cropped = None
max_bbox = None
for i, path in enumerate(path_list):
    image = cv2.imread(path)
    bbox = object_detection.detect_object_with_opencv(image, vis = False)
    croped_image = bbox_utils.crop_bbox_cv2(image,bbox)
    score, mask = comparison_score(croped_image, croped_image_scene)
    if score > max_score:
        max_ind = i
        max_mask = mask
        cropped = croped_image 
        max_score = score
        max_bbox = bbox



In [ ]:
image.shape[:2]
x_min,y_min, x_max,y_max = max_bbox
initial_mask = np.zeros(image.shape[:2])
initial_mask[y_min:y_max+1,x_min:x_max+1] = max_mask


In [ ]:
initial_mask.astype("int32")

In [ ]:
cropped[max_mask] = [255, 255, 255]
plt.imshow(cropped)

In [ ]:
np.save(data_dir/"mask.npy", mask)


In [ ]:
depth = o3d.io.read_image(depth_path)
type(depth)

In [ ]:
np.asarray(depth)

In [ ]:
o3d.io.write_image(data_dir/"masked_depth.png",o3d.geometry.Image(np.asarray(depth)[initial_mask.astype("int32")]))


In [ ]:
render_dir = data_dir/"renders"
render_iter = render_dir.glob("depth_*")
path_list= list(render_iter)
len(path_list)

In [ ]:
depth_path = path_list[max_ind]